#installing dependencies

# New Section

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [ ]:
!pip install optuna==2.3.0
!pip install transformers==4.2.1
!pip install farasapy
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert

     |████████████████████████████████| 258 kB 5.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81 kB 8.9 MB/s 
     |████████████████████████████████| 210 kB 40.6 MB/s 
     |████████████████████████████████| 78 kB 6.9 MB/s 
     |████████████████████████████████| 113 kB 40.8 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
     |████████████████████████████████| 150 kB 48.3 MB/s 
  Created wheel for optuna: filename=optuna-2.3.0-py3-none-any.whl size=359772 sha256=c15793c56dc2afae2b3aa06309ea3a52ba7d9581afe467a454bfbbb308f44fc4
  Stored in directory: /root/.cache/pip/wheels/38/61/9e/955ab1890f6cab231b1d756db63f36c711968a324296e0b649
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=2d2eacea1671f6e40e008cb477345af7a29b495ddbe1ae12c73d4090dd9b34a8
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f

In [ ]:
!mkdir data
!mkdir train

#Creating training datasets

In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
all_datasets= []

In [ ]:
class Dataset:
    def __init__(
        self,
        name,
        train,
        test,
        label_list,
    ):
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list

In [ ]:
DATA_COLUMN = "Tweets"
LABEL_COLUMN = "HS/NOT_HS"

##HARD - Balanced

In [ ]:
df_train=pd.read_csv("/content/OSACT2022_training set.csv")
df_valid=pd.read_csv("/content/OSACT2022_dev.csv")

In [ ]:
df_valid

,Tweets,HS/NOT_HS
0,افطرت عليك بعقاء واثنين من فروخها الجن,NOT_HS
1,داليا مبارك مادري ليش تقرفت,NOT_HS
2,ابديت السناب الجديد حاس الناس حوس أشوف مشاهير ...,NOT_HS
3,هييه والله وااايدد,NOT_HS
4,اكيد اخس شي,NOT_HS
...,...,...
1265,روما محظوظين بذا المدرب بيروتي يسحب في رجوله ا...,NOT_HS
1266,هلا والله بالحب هلا لولو من جد مو طايقتهم قللل...,NOT_HS
1267,رينز فاز يعنني اوه شوفو العرض الأسطوري تفو علي...,NOT_HS
1268,ييييع والله شيء يلوع الكبد مريضات الله يشفيهم,NOT_HS


In [ ]:
df_valid['HS/NOT_HS'].unique()

array(['NOT_HS', 'HS3', 'HS6', 'HS1', 'HS2', 'HS5', 'HS4'], dtype=object)

In [ ]:
df_train[LABEL_COLUMN]=df_train[LABEL_COLUMN].map({'NOT_HS':0,'HS1':1,'HS2':2,'HS3':3,'HS4':4,'HS5':5,'HS6':6})
df_valid[LABEL_COLUMN]=df_valid[LABEL_COLUMN].map({'NOT_HS':0,'HS1':1,'HS2':2,'HS3':3,'HS4':4,'HS5':5,'HS6':6})

In [ ]:
# train_HARD, test_HARD = train_test_split(df_HARD, test_size=0.2, random_state=42)
label_list_HARD = [0,1,2,3,4,5,6]
data_Hard = Dataset("HARD", df_train, df_valid, label_list_HARD)
all_datasets.append(data_Hard)

#Trainer

In [ ]:
from arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample
import logging
import torch
import optuna

In [ ]:
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

In [ ]:
for x in all_datasets:
  print(x.name)

HARD


You can choose which model, and dataset from here along with the max sentence length

In [ ]:
dataset_name = 'HARD'
model_name = 'aubmindlab/bert-base-arabertv02'
task_name = 'classification'
max_len = 256

In [ ]:
for d in all_datasets:
  if d.name==dataset_name:
    selected_dataset = d
    print('Dataset found')
    break

Dataset found


In [ ]:
selected_dataset.train

,Tweets,HS/NOT_HS
0,ردينا ع التطنز,0
1,وصارت فطاير البقالات غذاء صحي,0
2,روحي لبريده تلقين اشباه كثير بس ماحد زيكم مشفو...,0
3,مش باين حاجه خالص مش عارف بقى,0
4,اليوم الاثنين يقولك من المسلمين عندهم خاله اسم...,0
...,...,...
8882,الله يلعنهم,0
8883,واحد سال زوجته بعد كم سنة زواج حبيبتي كم رجال ...,0
8884,يالله روح زي الشاطر واحذف الشو الي سويته بس عش...,0
8885,لما الكلاب تهوهو عليك وأنت اللى مربيها,0


In [ ]:
arabert_prep = ArabertPreprocessor(model_name.split("/")[-1])

selected_dataset.train[DATA_COLUMN] = selected_dataset.train[DATA_COLUMN].apply(lambda x:   arabert_prep.preprocess(x))
selected_dataset.test[DATA_COLUMN] = selected_dataset.test[DATA_COLUMN].apply(lambda x:   arabert_prep.preprocess(x))

In [ ]:
selected_dataset.train

,Tweets,HS/NOT_HS
0,ردينا ع التطنز,0
1,وصارت فطاير البقالات غذاء صحي,0
2,روحي لبريده تلقين اشباه كثير بس ماحد زيكم مشفو...,0
3,مش باين حاجه خالص مش عارف بقى,0
4,اليوم الاثنين يقولك من المسلمين عندهم خاله اسم...,0
...,...,...
8882,الله يلعنهم,0
8883,واحد سال زوجته بعد كم سنة زواج حبيبتي كم رجال ...,0
8884,يالله روح زي الشاطر واحذف الشو الي سويته بس عش...,0
8885,لما الكلاب تهوهو عليك وأنت اللى مربيها,0


In [ ]:
test_set =pd.read_csv("/content/Test.csv")["Tweets"].apply(lambda x:   arabert_prep.preprocess(x))
len(test_set)

2541

In [ ]:
selected_dataset.test[DATA_COLUMN]

0                  افطرت عليك بعقاء واثنين من فروخها الجن
1                             داليا مبارك مادري ليش تقرفت
2       ابديت السناب الجديد حاس الناس حوس أشوف مشاهير ...
3                                       هييه والله واايدد
4                                             اكيد اخس شي
                              ...                        
1265    روما محظوظين بذا المدرب بيروتي يسحب في رجوله ا...
1266    هلا والله بالحب هلا لولو من جد مو طايقتهم قللع...
1267    رينز فاز يعنني اوه شوفو العرض الأسطوري تفو علي...
1268          ييع والله شيء يلوع الكبد مريضات الله يشفيهم
1269              تحسينها ع كليجه م اكلت شي واضح من الصوت
Name: Tweets, Length: 1270, dtype: object

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

In [ ]:
class embDataset(Dataset):
    def __init__(self, text, model_name, max_len):
      super(BERTDataset).__init__()
      self.text = text
      # self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      # self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask)

In [ ]:
# test_set= embDataset()
len(test_set)

2541

In [ ]:
model_name

'aubmindlab/bert-base-arabertv02'

In [ ]:
label_map = { v:index for index, v in enumerate(selected_dataset.label_list) }
print(label_map)
train_dataset = BERTDataset(selected_dataset.train[DATA_COLUMN].to_list(),selected_dataset.train[LABEL_COLUMN].to_list(),model_name,max_len,label_map)
valid_dataset = BERTDataset(selected_dataset.test[DATA_COLUMN].to_list(),selected_dataset.test[LABEL_COLUMN].to_list(),model_name,max_len,label_map)


{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6}


Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/825k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/381 [00:00<?, ?B/s]

In [ ]:
test_set = embDataset(test_set,model_name,max_len)
len(valid_dataset)

1270

In [ ]:
print(test_set)

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [ ]:
model_init()

Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:

Predictions = []
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  Predictions.append(preds)
  #print(classification_report(p.label_ids,preds))
  #print(confusion_matrix(p.label_ids,preds))
  macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[0,1])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {
      'macro_f1' : macro_f1,
      'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
  }

#Regular Training

This paert allows you to do a regular training with no hyper parameter optimization

In [ ]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate =5e-5
training_args.fp16 = True
training_args.per_device_train_batch_size = 16
training_args.per_device_eval_batch_size = 16
training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 5


steps_per_epoch = (len(selected_dataset.train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)
#Warmup_ratio
warmup_ratio = 0.1
training_args.warmup_steps = total_steps*warmup_ratio # or you can set the warmup steps directly 

training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000 #don't want to save any model, there is probably a better way to do this :)
training_args.seed = 21
training_args.disable_tqdm = False
training_args.lr_scheduler_type = 'cosine'

277
1385


In [ ]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Macro F1,Macro F1 Pos Neg,Macro Precision,Macro Recall,Accuracy,Runtime,Samples Per Second
1,No log,0.459758,0.789717,0.789717,0.781227,0.809686,0.807087,72.891200,17.423000
2,0.430200,0.414663,0.814626,0.814626,0.815046,0.814211,0.839370,72.871000,17.428000
3,0.430200,0.499330,0.804726,0.804726,0.802321,0.807365,0.829134,72.908100,17.419000
4,0.187700,0.747006,0.810954,0.810954,0.809960,0.811984,0.835433,72.876200,17.427000
5,0.187700,0.764617,0.813389,0.813389,0.817808,0.809506,0.840157,72.908400,17.419000


TrainOutput(global_step=1390, training_loss=0.2403996584226759, metrics={'train_runtime': 7756.5829, 'train_samples_per_second': 0.179, 'total_flos': 9227342717445120, 'epoch': 5.0})

In [ ]:
result=trainer.predict(test_set)


In [ ]:
print(result[0])

[[ 1.414   -1.06   ]
 [ 1.167   -1.057  ]
 [ 0.947   -0.9546 ]
 ...
 [ 1.128   -0.6675 ]
 [ 0.0917   0.2    ]
 [ 0.02257  0.609  ]]


In [ ]:
trainer.save_model("OFF_Not_OFF")

In [ ]:
def conv2txt(labels):
  txtLabels = []
  for i in labels:
    val, idx = max((val, idx) for (idx, val) in enumerate(i))
    if idx == 0:
      txtLabels.append("NOT_OFF")
    elif idx == 1:
      txtLabels.append("OFF")
  return txtLabels

In [ ]:
from google.colab import files
labels= conv2txt(result[0])
with open("labels_TaskA.txt", 'w') as fd:
      for lbl in labels:
        fd.write(lbl + "\n")
files.download('labels_TaskA.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
! zip -r /content/OFF_Not_OFF.zip /content/OFF_Not_OFF

  adding: content/OFF_Not_OFF/ (stored 0%)
  adding: content/OFF_Not_OFF/training_args.bin (deflated 44%)
  adding: content/OFF_Not_OFF/config.json (deflated 46%)
  adding: content/OFF_Not_OFF/pytorch_model.bin (deflated 7%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
shutil.move('/content/OFF_Not_OFF.zip','/content/drive/MyDrive/')

In [ ]:
import shutil
shutil.copy('/content/drive/MyDrive/OFF_Not_OFF.zip','/content')

'/content/OFF_Not_OFF.zip'

In [ ]:
from zipfile import ZipFile
with ZipFile('OFF_Not_OFF.zip', 'r') as zip:
	# printing all the contents of the zip file
	zip.printdir()

	# extracting all the files
	print('Extracting all the files now...')
	zip.extractall()
	print('Done!')

File Name                                             Modified             Size
content/OFF_Not_OFF/                           2022-03-25 21:24:36            0
content/OFF_Not_OFF/training_args.bin          2022-03-25 21:24:36         1967
content/OFF_Not_OFF/config.json                2022-03-25 21:24:34          633
content/OFF_Not_OFF/pytorch_model.bin          2022-03-25 21:24:36    540863661
Extracting all the files now...
Done!


In [ ]:
!unzip '/content/drive/MyDrive/OFF_Not_OFF.zip'

Archive:  /content/drive/MyDrive/OFF_Not_OFF.zip
replace content/OFF_Not_OFF/training_args.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: content/OFF_Not_OFF/training_args.bin  
replace content/OFF_Not_OFF/config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: content/OFF_Not_OFF/config.json  
replace content/OFF_Not_OFF/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: content/OFF_Not_OFF/pytorch_model.bin  y



In [ ]:
import torch 
modell= torch.load('/content/content/rename/pytorch_model.bin')
print(modell)

<built-in method values of collections.OrderedDict object at 0x7fed541bd200>


In [ ]:
modell.load_state_dict(modell)‏

In [ ]:
trainer = Trainer(
    model = modell(),
    args = '/content/content/rename/training_args.bin',
    train_dataset = train_dataset,
    eval_dataset='/content/Test.csv',
    compute_metrics=compute_metrics,
)